<a href="https://colab.research.google.com/github/alexgill321/EKFAC-Influence-Benchmarks/blob/main/EKFAC_calculate_if_value.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn.datasets import load_breast_cancer
data = load_breast_cancer()
x = data['data']
y = data['target']
print("shape of x: {}\nshape of y: {}".format(x.shape,y.shape))

shape of x: (569, 30)
shape of y: (569,)


In [2]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size = 0.25, random_state = 0)

In [46]:

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)


from torch.utils.data import Dataset, DataLoader
import torch

class dataset(Dataset):
  def __init__(self,x,y):
    self.x = torch.tensor(x,dtype=torch.float32)
    self.y = torch.tensor(y,dtype=torch.float32)
    self.length = self.x.shape[0]

  def __getitem__(self,idx):
    return self.x[idx],self.y[idx]
  def __len__(self):
    return self.length

trainset = dataset(X_train,Y_train)
trainloader = DataLoader(trainset,batch_size=60,shuffle=False)



valset = dataset(X_test,Y_test)
valloader = DataLoader(valset,batch_size=60,shuffle=False)

In [47]:
from torch import nn
from torch.nn import functional as F
class Net(nn.Module):
  def __init__(self,input_shape):
    super(Net,self).__init__()
    self.fc1 = nn.Linear(input_shape,32)
    self.fc2 = nn.Linear(32,64)
    self.fc3 = nn.Linear(64,1)
  def forward(self,x):
    x = torch.relu(self.fc1(x))
    x = torch.relu(self.fc2(x))
    x = self.fc3(x)
    return x


In [48]:
from torch import optim
network = Net(x.shape[1])
optimizer = optim.Adam(network.parameters(), lr=0.001)


In [49]:
from torch.optim.optimizer import Optimizer
class EKFACDistilled(Optimizer):
    def __init__(self, net, eps):
        self.eps = eps
        self.params = []
        self._fwd_handles = []
        self._bwd_handles = []
        self.net = net
        for mod in net.modules():
          mod_class = mod.__class__.__name__
          if mod_class in ['Linear']:
              handle = mod.register_forward_pre_hook(self._save_input)
              self._fwd_handles.append(handle)
              handle = mod.register_full_backward_hook(self._save_grad_output)
              self._bwd_handles.append(handle)
              params = [mod.weight]
              if mod.bias is not None:
                  params.append(mod.bias)
              d = {'params': params, 'mod': mod, 'layer_type': mod_class}
              self.params.append(d)
        super(EKFACDistilled, self).__init__(self.params, {})

    def step(self, update_stats=True, update_params=True):
        for group in self.param_groups:
            if len(group['params']) == 2:
                weight, bias = group['params']
            else:
                weight = group['params'][0]
                bias = None
            state = self.state[weight]

            self._compute_kfe(group, state)

            self._precond(weight, bias, group, state)

    def _compute_kfe(self, group, state):
        mod = group['mod']
        x = self.state[group['mod']]['x']
        print(f"Shape of x: {x.shape}")
        gy = self.state[group['mod']]['gy']
        print(f"Shape of gy: {gy.shape}")

        # Computation of xxt
        x = x.data.t() # transpose of activations

        # Append column of ones to x if bias is not None
        if mod.bias is not None:
            ones = torch.ones_like(x[:1])
            x = torch.cat([x, ones], dim=0)

        # Calculate covariance matrix for activations (A_{l-1})
        xxt = torch.mm(x, x.t()) / float(x.shape[1])

        print(f'A cov matrix shape: {xxt.shape}')

        # Calculate eigenvalues and eigenvectors of covariance matrix (lambdaA, QA)
        la, Qa = torch.linalg.eigh(xxt, UPLO='U')
        state['Qa'] = Qa
        print(f'Qa eigenvec shape: {Qa.shape}')
        print(f'LambdaA eigenval vec shape: {la.shape}')
        # Computation of ggt
        gy = gy.data.t()

        # Calculate covariance matrix for layer outputs (S_{l})
        ggt = torch.mm(gy, gy.t()) / float(gy.shape[1])

        print(f'S cov matrix shape: {ggt.shape}')
        # Calculate eigenvalues and eigenvectors of covariance matrix (lambdaS, QS)
        ls, Qs = torch.linalg.eigh(ggt, UPLO='U')

        G_real = torch.kron(xxt,ggt)
        print(f'AxS direct shape: {G_real.shape}')

        state['Qs'] = Qs

        print(f'Qs eigenvec shape: {Qs.shape}')
        print(f'LambdaS eigenval vec shape: {ls.shape}')

        prod_as = torch.kron(Qa, Qs)

        print(f'Kroneker product of Qa * Qs: {prod_as.shape}')

        prod_eigval = torch.kron(torch.diag(la),torch.diag(ls))

        G = torch.matmul(prod_as,torch.matmul(prod_eigval, prod_as.t()))

        print(f'G SHAPE: {G.shape}')

        print(f'Kroneker product of LambdaA * LambdaS: {prod_eigval.shape}')

        # Outer product of the eigenvalue vectors. Of shape (len(s) x len(a))
        state["m2"] = m2 = ls.unsqueeze(1) * la.unsqueeze(0)
        print(f"eigenval outer product shape: {m2.shape}")

        print(G_real - G)

    def _precond(self, weight, bias, group, state):
        """Applies preconditioning."""
        Qa = state['Qa']
        Qs = state['Qs']
        m2 = state['m2']
        x = self.state[group['mod']]['x']
        print(x)
        gy = self.state[group['mod']]['gy']
        g = weight.grad.data
        s = g.shape
        s_x = x.size()
        s_gy = gy.size()
        bs = x.size(0)

        # Append column of ones to x if bias is not None
        if bias is not None:
            ones = torch.ones_like(x[:,:1])
            x = torch.cat([x, ones], dim=1)

        # KFE of activations ??
        x_kfe = torch.mm(x, Qa)

        print(f"KFE of activations a shape: {x_kfe.shape}")

        # KFE of layer outputs ??
        gy_kfe = torch.mm(gy, Qs)

        print(f"KFE of outputs gy shape: {gy_kfe.shape}")

        m2 = torch.mm(gy_kfe.t()**2, x_kfe**2) / bs

        print(f'kfe squared matrix idk shape: {m2.shape}')
        g_kfe = torch.mm(gy_kfe.t(), x_kfe) / bs

        print(f'g_kfe shape: {g_kfe.shape}')

        g_nat_kfe = g_kfe / (m2 + self.eps)

        print(f'g_nat_kfe shape: {g_nat_kfe.shape}')

        g_nat = torch.mm(g_nat_kfe, Qs.t())

        if bias is not None:
            gb = g_nat[:, -1].contiguous().view(*bias.shape)
            bias.grad.data = gb
            g_nat = g_nat[:, :-1]

        g_nat = g_nat.contiguous().view(*s)
        weight.grad.data = g_nat

    def _save_input(self, mod, i):
        """Saves input of layer to compute covariance."""
        self.state[mod]['x'] = i[0]

    def _save_grad_output(self, mod, grad_input, grad_output):
        """Saves grad on output of layer to compute covariance."""
        self.state[mod]['gy'] = grad_output[0] * grad_output[0].size(0)


In [50]:
precond = EKFACDistilled(network, eps=0.001)
criterion = torch.nn.BCELoss()

for mod in network.modules():
  mod_class = mod.__class__.__name__
  print(mod_class)
  print("**********************")

Net
**********************
Linear
**********************
Linear
**********************
Linear
**********************


# **Create a score matrix from the query (test datapoint)**

In [51]:
# just need one exmanple from the test set as a "QUERY"


query_influence_tensor = ''

for i, (inputs, targets) in enumerate(valloader):
  optimizer.zero_grad()
  outputs = network(inputs)
  # print(outputs[0].backward())
  outputs[0].backward()

  grad_wrt_logits_weights = network.fc1.weight.grad
  grad_wrt_logits_bias = network.fc1.bias.grad

  print("the linear layer weight grad wrt the output: {}".format(grad_wrt_logits_weights.shape)) #32, 30
  print("the linear layer bias grad wrt the output: {}".format(grad_wrt_logits_bias.shape)) #32

  grad_wrt_logits_bias = grad_wrt_logits_bias.view(32, 1)

  total_linear_layer_params = torch.cat((grad_wrt_logits_weights, grad_wrt_logits_bias), dim=1)

  query_influence_tensor = total_linear_layer_params.view(-1)
  print(query_influence_tensor.shape)

  break

the linear layer weight grad wrt the output: torch.Size([32, 30])
the linear layer bias grad wrt the output: torch.Size([32])
torch.Size([992])


# **Create the score matrix for each training example (or a random subset of examples), this will be mutlitpled with `query_influence_tensor` and `G`**

In [80]:
training_example_influence_tensors = {}



# we will take the first example from each batch.

for i, (inputs, targets) in enumerate(trainloader):
  optimizer.zero_grad()
  outputs = network(inputs)
  probabilities = torch.sigmoid(outputs)

  loss = criterion(probabilities, targets.view(-1,1))
  loss.backward()


  layer_grad_wrt_loss_weights = network.fc1.weight.grad
  layer_grad_wrt_loss_bias = network.fc1.bias.grad

  # print("the linear layer weight grad wrt the loss: {}".format(layer_grad_wrt_loss_weights.shape)) #32,30
  # print("the linear layer bias grad wrt the loss: {}".format(layer_grad_wrt_loss_bias.shape)) #32

  layer_grad_wrt_loss_bias = layer_grad_wrt_loss_bias.view(32, 1)


  total_linear_layer_params = torch.cat((layer_grad_wrt_loss_weights, layer_grad_wrt_loss_bias), dim=1)
  training_influence_tensor = total_linear_layer_params.view(992)

  training_example_influence_tensors[i] = training_influence_tensor

print('total examples from training set are: {}'.format(len(training_example_influence_tensors)))

total examples from training set are: 8


# **Now calculate IF for each example (8 total, one from each batch)**/


In [89]:
# taking a temp gaussian approximation vector
approx_gaussian = torch.randn(992, 992)


influence_values = []

for key, value in training_example_influence_tensors.items():
  val = torch.matmul(value,torch.matmul(approx_gaussian, query_influence_tensor).view(992, 1))
  print(val)
  influence_values.append(val.item())


print("highest training influence example is example 0 form batch {} with value {}".format(influence_values.index(max(influence_values)), max(influence_values)))

tensor([0.0296])
tensor([0.0775])
tensor([0.1017])
tensor([0.1043])
tensor([0.0847])
tensor([0.0820])
tensor([0.0787])
tensor([0.1452])
highest training influence example is example 0 form batch 7 with value 0.1451510787010193
